In [1]:
import numpy as np
import pandas as pd
import os 
import sys
import glob
import time
import warnings
import matplotlib.pyplot as plt
from matplotlib import colors, ticker
%matplotlib notebook

# load static copy of modules for reproducibility
if '../../bayes-drt_20201113' not in sys.path:
    sys.path.append('../../bayes-drt_20201113')
import drt
from stan_models import save_pickle,load_pickle
import eis_utils as gt

%load_ext autoreload
%autoreload 2

In [2]:
datadir = '../../../data'
outdir = 'results'

# Simulated data

## Basic circuits

In [3]:
dr = drt.DRT()
files = glob.glob('../../../data/simulated/Z_*Orazem*.csv')
# trunc and RC-ZARC circuits need different basis tau
files = [f for f in files if f.find('trunc')==-1 and f.find('RC-ZARC')==-1 and f.find('DDT')==-1]
# assign epsilon values for circuits
circ_epsilon = {'ZARC':2,'2ZARC':2,'RC-ZARC':5,'ZARC-RL':3,'RC':5,'2RC':5,'Gerischer':5}

In [4]:
# Fit all basic simulated spectra 

# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

for file in files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):-4]
    
#     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#         print('Already ran')
#     else:
    file_start = time.time()

    # get circuit-specific epsilon
    circ = os.path.split(file)[1].split('_')[1]
    epsilon = circ_epsilon[circ]
    dr.epsilon = epsilon
    # set nonneg=False for ZARC-RL
    if circ=='ZARC-RL':
        nonneg = False
    else:
        nonneg = True
        
    # get error model and set weighting scheme accordingly
    err_model = os.path.split(file)[1].split('_')[2]
    if err_model=='uniform':
        weights='unity'
    elif err_model=='Macdonald':
        weights='prop_adj'
    elif err_model=='Orazem':
        weights='Orazem'
    # use unity weighting for all spectra as done by Ciucci and Chen (2015)
#     weights = 'unity'

    # load Z data
    Zndf = pd.read_csv(file)
    freq = Zndf['Freq'].values
    Zc = (Zndf['Zreal'] + 1j*Zndf['Zimag']).values

    # perform Re-Im CV
    fit_kw = dict(reg_ord=2,dZ=False,scale_Z=False,nonneg=nonneg,weights=weights,
                  penalty='discrete',max_iter=50)       
    cv_lambdas = np.logspace(-15,0,61)
    dr.ridge_fit(freq,Zc,lambda_0='cv',cv_lambdas=cv_lambdas,hyper_lambda=False,**fit_kw)

    # get best lambda_0
    min_lam = dr.cv_result['lambda'].values[np.argmin(dr.cv_result['totcv'])]
    if min_lam==np.min(cv_lambdas) or min_lam==np.max(cv_lambdas):
        warnings.warn('min_lam value {} for file {} is at lambda boundary!'.format(min_lam,os.path.split(file)[1]))

    # run hyperparametric fits with best lambda_0
    f_betas = [1e-2,1e-1,1,1e1,1e2]
    for f_beta in f_betas:
        Zoutfile = os.path.join(outdir,'Zout{}_fbeta={:.3g}.csv'.format(suffix,f_beta))
        Goutfile = os.path.join(outdir,'Gout{}_fbeta={:.3g}.csv'.format(suffix,f_beta))
        pkl = os.path.join(outdir,'obj{}_fbeta={:.3g}.pkl'.format(suffix,f_beta))

        dr.ridge_fit(freq,Zc,part='both',lambda_0=min_lam,hl_fbeta=f_beta,**fit_kw)

        # save results
        Z_pred = dr.predict_Z(freq)
        Z_res = pd.DataFrame(np.array([freq,Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])

        g_pred = dr.predict_distribution('DRT',eval_tau=tau_plot)
        g_res = pd.DataFrame(np.array([tau_plot,g_pred]).T,
                             columns=['tau','gamma'])
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

        # pickle the drt object for access to all other attributes
        save_pickle(dr,pkl)

    print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        
        
        
print('================================')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../../../data/simulated\Z_2RC_Orazem_0.25.csv
-------------------------------


INFO:numexpr.utils:NumExpr defaulting to 4 threads.


File fit time: 28.91 seconds
Total fit time: 0.48 minutes


## RC-ZARC circuit

In [307]:
basis_tau = np.logspace(-2,3,51)
dr_rcz = drt.DRT(basis_freq=1/(2*np.pi*basis_tau),epsilon=5)
rcz_files = glob.glob(os.path.join(datadir,'simulated/Z_RC-ZARC*.csv'))

In [407]:
# Fit RC-ZARC spectra

# tau for plotting
tau_plot = np.logspace(np.log10(np.exp(-5)),np.log10(np.exp(5.5)),200)
start = time.time()

for file in rcz_files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):-4]
    
#     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#         print('Already ran')
#     else:
    file_start = time.time()

    nonneg = True
        
    # get error model and set weighting scheme accordingly
    err_model = os.path.split(file)[1].split('_')[2]
    if err_model=='uniform':
        weights='unity'
    elif err_model=='Macdonald':
        weights='prop_adj'
    elif err_model=='Orazem':
        weights='Orazem'

    # load Z data
    Zndf = pd.read_csv(file)
    freq = Zndf['Freq'].values
    Zc = (Zndf['Zreal'] + 1j*Zndf['Zimag']).values

    # perform Re-Im CV
    fit_kw = dict(reg_ord=2,dZ=False,scale_Z=False,nonneg=nonneg,weights=weights,
                  penalty='discrete',max_iter=50)       
    dr_rcz.ridge_fit(freq,Zc,lambda_0='cv',cv_lambdas=np.logspace(-10,0,41),hyper_lambda=False,**fit_kw)
    
    # get best lambda_0
    min_lam = dr_rcz.cv_result['lambda'].values[np.argmin(dr_rcz.cv_result['totcv'])]
    if min_lam==np.min(lambdas) or min_lam==np.max(lambdas):
        warnings.warn('min_lam value {} for file {} is at lambda boundary!'.format(min_lam,os.path.split(file)[1]))

    # run hyperparametric fits with best lambda_0
    f_betas = [1e-2,1e-1,1,1e1,1e2]
    for f_beta in f_betas:
        Zoutfile = os.path.join(outdir,'Zout{}_fbeta={:.3g}.csv'.format(suffix,f_beta))
        Goutfile = os.path.join(outdir,'Gout{}_fbeta={:.3g}.csv'.format(suffix,f_beta))
        pkl = os.path.join(outdir,'obj{}_fbeta={:.3g}.pkl'.format(suffix,f_beta))

        dr_rcz.ridge_fit(freq,Zc,part='both',lambda_0=min_lam,hl_fbeta=f_beta,**fit_kw)

        # save results
        Z_pred = dr_rcz.predict_Z(freq)
        Z_res = pd.DataFrame(np.array([freq,Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])

        g_pred = dr_rcz.predict_distribution('DRT',eval_tau=tau_plot)
        g_res = pd.DataFrame(np.array([tau_plot,g_pred]).T,
                             columns=['tau','gamma'])
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

        # pickle the drt object for access to all other attributes
        save_pickle(dr_rcz,pkl)

    print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        
        
print('================================')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../../../data\simulated\Z_RC-ZARC_Macdonald_0.25.csv
-------------------------------
Dumped pickle to results\obj_RC-ZARC_Macdonald_0.25_fbeta=0.01.pkl
Dumped pickle to results\obj_RC-ZARC_Macdonald_0.25_fbeta=0.1.pkl
Dumped pickle to results\obj_RC-ZARC_Macdonald_0.25_fbeta=1.pkl
Dumped pickle to results\obj_RC-ZARC_Macdonald_0.25_fbeta=10.pkl
Dumped pickle to results\obj_RC-ZARC_Macdonald_0.25_fbeta=100.pkl
File fit time: 8.93 seconds
-------------------------------
../../../data\simulated\Z_RC-ZARC_Macdonald_1.0.csv
-------------------------------
Dumped pickle to results\obj_RC-ZARC_Macdonald_1.0_fbeta=0.01.pkl
Dumped pickle to results\obj_RC-ZARC_Macdonald_1.0_fbeta=0.1.pkl
Dumped pickle to results\obj_RC-ZARC_Macdonald_1.0_fbeta=1.pkl
Dumped pickle to results\obj_RC-ZARC_Macdonald_1.0_fbeta=10.pkl
Dumped pickle to results\obj_RC-ZARC_Macdonald_1.0_fbeta=100.pkl
File fit time: 7.96 seconds
-------------------------------
../../../data\simulated\Z_RC

## trunc circuit

In [34]:
dr_trunc = drt.DRT(basis_freq=np.logspace(6,-2,81),epsilon=3)
trunc_files = glob.glob(os.path.join(datadir,'simulated/Z_trunc*.csv'))

In [38]:
# Fit trunc spectra

# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

for file in trunc_files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):-4]
    
#     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#         print('Already ran')
#     else:
    file_start = time.time()

    nonneg = False
        
    # get error model and set weighting scheme accordingly
    err_model = os.path.split(file)[1].split('_')[2]
    if err_model=='uniform':
        weights='unity'
    elif err_model=='Macdonald':
        weights='prop_adj'
    elif err_model=='Orazem':
        weights='Orazem'

    # load Z data
    Zndf = pd.read_csv(file)
    freq = Zndf['Freq'].values
    Zc = (Zndf['Zreal'] + 1j*Zndf['Zimag']).values

    # perform Re-Im CV
    fit_kw = dict(reg_ord=2,dZ=False,scale_Z=False,nonneg=nonneg,weights=weights,
                  penalty='discrete',max_iter=50)       
    lambdas = np.logspace(-10,0,41)
    dr_trunc.ridge_fit(freq,Zc,lambda_0='cv',cv_lambdas=lambdas,hyper_lambda=False,**fit_kw)
    
    # get best lambda_0
    min_lam = dr_trunc.cv_result['lambda'].values[np.argmin(dr_trunc.cv_result['totcv'])]
    if min_lam==np.min(lambdas) or min_lam==np.max(lambdas):
        warnings.warn('min_lam value {} for file {} is at lambda boundary!'.format(min_lam,os.path.split(file)[1]))

    # run hyperparametric fits with best lambda_0
    f_betas = [1e-2,1e-1,1,1e1,1e2]
    for f_beta in f_betas:
        Zoutfile = os.path.join(outdir,'Zout{}_fbeta={:.3g}.csv'.format(suffix,f_beta))
        Goutfile = os.path.join(outdir,'Gout{}_fbeta={:.3g}.csv'.format(suffix,f_beta))
        pkl = os.path.join(outdir,'obj{}_fbeta={:.3g}.pkl'.format(suffix,f_beta))

        dr_trunc.ridge_fit(freq,Zc,part='both',lambda_0=min_lam,hl_fbeta=f_beta,**fit_kw)

        # save results
        Z_pred = dr_trunc.predict_Z(freq)
        Z_res = pd.DataFrame(np.array([freq,Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])

        g_pred = dr_trunc.predict_distribution('DRT',eval_tau=tau_plot)
        g_res = pd.DataFrame(np.array([tau_plot,g_pred]).T,
                             columns=['tau','gamma'])
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

        # pickle the drt object for access to all other attributes
        save_pickle(dr_trunc,pkl)

    print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        
        
print('================================')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../../../data\simulated\Z_trunc_Macdonald_0.25.csv
-------------------------------


../../../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 50 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


Dumped pickle to results\obj_trunc_Macdonald_0.25_fbeta=0.01.pkl


../../../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 50 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


Dumped pickle to results\obj_trunc_Macdonald_0.25_fbeta=0.1.pkl
Dumped pickle to results\obj_trunc_Macdonald_0.25_fbeta=1.pkl
Dumped pickle to results\obj_trunc_Macdonald_0.25_fbeta=10.pkl
Dumped pickle to results\obj_trunc_Macdonald_0.25_fbeta=100.pkl
File fit time: 29.71 seconds
-------------------------------
../../../data\simulated\Z_trunc_Macdonald_1.0.csv
-------------------------------


...

Dumped pickle to results\obj_trunc_uniform_0.25_fbeta=0.1.pkl
Dumped pickle to results\obj_trunc_uniform_0.25_fbeta=1.pkl
Dumped pickle to results\obj_trunc_uniform_0.25_fbeta=10.pkl
Dumped pickle to results\obj_trunc_uniform_0.25_fbeta=100.pkl
File fit time: 33.58 seconds
-------------------------------
../../../data\simulated\Z_trunc_uniform_1.0.csv
-------------------------------


../../../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 50 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


Dumped pickle to results\obj_trunc_uniform_1.0_fbeta=0.01.pkl


../../../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 50 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


Dumped pickle to results\obj_trunc_uniform_1.0_fbeta=0.1.pkl
Dumped pickle to results\obj_trunc_uniform_1.0_fbeta=1.pkl
Dumped pickle to results\obj_trunc_uniform_1.0_fbeta=10.pkl
Dumped pickle to results\obj_trunc_uniform_1.0_fbeta=100.pkl
File fit time: 27.52 seconds
Total fit time: 2.68 minutes


## DDT

In [54]:
tp_files = glob.glob(os.path.join(datadir,'simulated/Z_*TP-DDT*.csv'))
bp_files = glob.glob(os.path.join(datadir,'../data/simulated/Z_*BP-DDT*.csv'))

tp_ddt = drt.DRT(distributions={'DDT':{'kernel':'DDT','symmetry':'planar','bc':'transmissive','dist_type':'parallel'}},
                epsilon=3)
bp_ddt = drt.DRT(distributions={'DDT':{'kernel':'DDT','symmetry':'planar','bc':'blocking','dist_type':'parallel',
                                      'basis_freq':np.logspace(5,-2,71)}},
                 epsilon=3
                )

In [57]:
tau_plot = np.logspace(-7,2,200)
start = time.time()

for files,dr in zip([tp_files,bp_files],[tp_ddt,bp_ddt]):
    for file in files:
        print('-------------------------------')
        print(file)
        print('-------------------------------')
        fname = os.path.split(file)[1]
        suffix = fname[fname.find('_'):-4]

    #     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
    #         print('Already ran')
    #     else:
        file_start = time.time()

        nonneg = True
        # modulus weighting works best for fitting admittance
        weights='modulus'

        # load Z data
        Zndf = pd.read_csv(file)
        freq = Zndf['Freq'].values
        Zc = (Zndf['Zreal'] + 1j*Zndf['Zimag']).values

        # perform Re-Im CV
        fit_kw = dict(reg_ord=2,dZ=False,scale_Z=False,nonneg=nonneg,weights=weights,
                      penalty='discrete',max_iter=50)      
        if fname=='Z_BimodalBP-DDT_uniform_0.25.csv':
            # this file needs higher lambda value
            lambdas = np.logspace(-5,5,41)
        else:
            lambdas = np.logspace(-10,0,41)
        min_lam = dr.ridge_ReImCV(freq,Zc,lambdas=lambdas,**fit_kw)
        if min_lam==np.min(lambdas) or min_lam==np.max(lambdas):
            warnings.warn('min_lam value {} for file {} is at lambda boundary!'.format(min_lam,fname))

        # run hyperparametric fits with best lambda_0
        f_betas = [1e-2,1e-1,1,1e1,1e2]
        for f_beta in f_betas:
            Zoutfile = os.path.join(outdir,'Zout{}_fbeta={:.3g}.csv'.format(suffix,f_beta))
            Goutfile = os.path.join(outdir,'Gout{}_fbeta={:.3g}.csv'.format(suffix,f_beta))
            pkl = os.path.join(outdir,'obj{}_fbeta={:.3g}.pkl'.format(suffix,f_beta))

            dr.ridge_fit(freq,Zc,part='both',lambda_0=min_lam,hl_fbeta=f_beta,**fit_kw)

            # save results
            Z_pred = dr.predict_Z(freq)
            Z_res = pd.DataFrame(np.array([freq,Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])

            g_pred = dr.predict_distribution('DDT',eval_tau=tau_plot)
            g_res = pd.DataFrame(np.array([tau_plot,g_pred]).T,
                                 columns=['tau','gamma'])
            Z_res.to_csv(Zoutfile,index=False)
            g_res.to_csv(Goutfile,index=False)

            # pickle the drt object for access to all other attributes
            save_pickle(dr,pkl)

        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

        
print('================================')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../../../data\simulated\Z_BimodalTP-DDT_Macdonald_0.05.csv
-------------------------------


../../../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 50 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


Dumped pickle to results\obj_BimodalTP-DDT_Macdonald_0.05_fbeta=0.01.pkl


../../../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 50 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


Dumped pickle to results\obj_BimodalTP-DDT_Macdonald_0.05_fbeta=0.1.pkl
Dumped pickle to results\obj_BimodalTP-DDT_Macdonald_0.05_fbeta=1.pkl
Dumped pickle to results\obj_BimodalTP-DDT_Macdonald_0.05_fbeta=10.pkl
Dumped pickle to results\obj_BimodalTP-DDT_Macdonald_0.05_fbeta=100.pkl
File fit time: 21.00 seconds
-------------------------------
../../../data\simulated\Z_BimodalTP-DDT_Macdonald_0.25.csv
-------------------------------


...

Dumped pickle to results\obj_BimodalBP-DDT_uniform_0.05_fbeta=0.1.pkl
Dumped pickle to results\obj_BimodalBP-DDT_uniform_0.05_fbeta=1.pkl
Dumped pickle to results\obj_BimodalBP-DDT_uniform_0.05_fbeta=10.pkl
Dumped pickle to results\obj_BimodalBP-DDT_uniform_0.05_fbeta=100.pkl
File fit time: 11.25 seconds
-------------------------------
../../../data\../data/simulated\Z_BimodalBP-DDT_uniform_0.25.csv
-------------------------------


../../../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 50 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


Dumped pickle to results\obj_BimodalBP-DDT_uniform_0.25_fbeta=0.01.pkl


../../../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 50 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


Dumped pickle to results\obj_BimodalBP-DDT_uniform_0.25_fbeta=0.1.pkl
Dumped pickle to results\obj_BimodalBP-DDT_uniform_0.25_fbeta=1.pkl
Dumped pickle to results\obj_BimodalBP-DDT_uniform_0.25_fbeta=10.pkl
Dumped pickle to results\obj_BimodalBP-DDT_uniform_0.25_fbeta=100.pkl
File fit time: 12.02 seconds
Total fit time: 3.63 minutes


# Experimental data

## LIB data

In [206]:
lib_files = ['DRTtools_LIB_data.txt','DRTtools_LIB_data_qtr.csv']
lib_files = [os.path.join(datadir,'experimental',f) for f in lib_files]
lib_suffixes = ['_LIB_data','_LIB_data_qtr']

lib_basis_freq = np.logspace(4,-5,91)
lib_tau = 1/(2*np.pi*lib_basis_freq)

dr_lib = drt.DRT(basis_freq=lib_basis_freq,epsilon=3)

In [209]:
dr = dr_lib
start = time.time()
tau_plot_lib = np.logspace(-5,4,200)
for file,suffix in zip(lib_files,lib_suffixes):
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    fname = os.path.split(file)[1]

#     if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
#         print('Already ran')
#     else:
    file_start = time.time()

    nonneg = True
    weights='unity'

    # load Z data
    if file[-3:]=='txt':
        Zndf = pd.read_csv(file,sep='\t',header=None)
        Zndf = pd.DataFrame(Zndf.values,columns=['Freq','Zreal','Zimag'])
    else:
        Zndf = pd.read_csv(file)
    freq = Zndf['Freq'].values
    Zc = (Zndf['Zreal'] + 1j*Zndf['Zimag']).values

    # perform Re-Im CV
    fit_kw = dict(reg_ord=2,dZ=False,scale_Z=False,nonneg=nonneg,weights=weights,
                  penalty='discrete',max_iter=50)      
    lambdas = np.logspace(-12,-2,41)
    min_lam = dr.ridge_ReImCV(freq,Zc,lambdas=lambdas,**fit_kw)
    if min_lam==np.min(lambdas) or min_lam==np.max(lambdas):
        warnings.warn('min_lam value {} for file {} is at lambda boundary!'.format(min_lam,fname))

    # run hyperparametric fits with best lambda_0
    f_betas = [1e-2,1e-1,1,1e1,1e2]
    for f_beta in f_betas:
        print(suffix)
        Zoutfile = os.path.join(outdir,'Zout{}_fbeta={:.3g}.csv'.format(suffix,f_beta))
        Goutfile = os.path.join(outdir,'Gout{}_fbeta={:.3g}.csv'.format(suffix,f_beta))
        pkl = os.path.join(outdir,'obj{}_fbeta={:.3g}.pkl'.format(suffix,f_beta))

        dr.ridge_fit(freq,Zc,part='both',lambda_0=min_lam,hl_fbeta=f_beta,**fit_kw)

        # save results
        Z_pred = dr.predict_Z(freq)
        Z_res = pd.DataFrame(np.array([freq,Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])

        g_pred = dr.predict_distribution('DRT',eval_tau=tau_plot_lib)
        g_res = pd.DataFrame(np.array([tau_plot_lib,g_pred]).T,
                             columns=['tau','gamma'])
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

        # pickle the drt object for access to all other attributes
        save_pickle(dr,pkl)

    print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

        
print('================================')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../../../data\experimental\DRTtools_LIB_data.txt
-------------------------------
_LIB_data


../../../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 50 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


Dumped pickle to results\obj_LIB_data_fbeta=0.01.pkl
_LIB_data


../../../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 50 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


Dumped pickle to results\obj_LIB_data_fbeta=0.1.pkl
_LIB_data
Dumped pickle to results\obj_LIB_data_fbeta=1.pkl
_LIB_data
Dumped pickle to results\obj_LIB_data_fbeta=10.pkl
_LIB_data
Dumped pickle to results\obj_LIB_data_fbeta=100.pkl
File fit time: 26.77 seconds
-------------------------------
../../../data\experimental\DRTtools_LIB_data_qtr.csv
-------------------------------
_LIB_data_qtr


../../../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 50 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


Dumped pickle to results\obj_LIB_data_qtr_fbeta=0.01.pkl
_LIB_data_qtr


../../../bayes-drt\drt.py:448: UserWarning: Hyperparametric solution did not converge within 50 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


Dumped pickle to results\obj_LIB_data_qtr_fbeta=0.1.pkl
_LIB_data_qtr
Dumped pickle to results\obj_LIB_data_qtr_fbeta=1.pkl
_LIB_data_qtr
Dumped pickle to results\obj_LIB_data_qtr_fbeta=10.pkl
_LIB_data_qtr
Dumped pickle to results\obj_LIB_data_qtr_fbeta=100.pkl
File fit time: 36.96 seconds
Total fit time: 1.06 minutes


In [211]:
dr.R_inf

0.10913792982407555

## Protonic ceramic microelectrode

In [173]:
tco_file = os.path.join(datadir,'experimental/PDAC_COM3_02109_Contact10_2065C_500C.txt')
suffix = '_PDAC'
tco_df = gt.read_eis_zdata(tco_file)
Zc = tco_df['Zreal'].values + 1j*tco_df['Zimag'].values
freq = tco_df['Freq'].values

dr_tco = drt.DRT(basis_freq=np.logspace(7,-3,101),epsilon=3)
# tau for plotting
tau_plot_tco = np.logspace(-7,4,200)

# perform Re-Im CV
# use scale_Z=True to get Z to regular scale - makes hyper-w tuning easier
fit_kw = dict(reg_ord=2,dZ=False,scale_Z=False,nonneg=True,weights='unity',
              penalty='discrete',max_iter=50)      
lambdas = np.logspace(-10,0,41)
min_lam = dr_tco.ridge_ReImCV(freq,Zc,lambdas=lambdas,**fit_kw)
if min_lam==np.min(lambdas) or min_lam==np.max(lambdas):
    warnings.warn('min_lam value {} for file {} is at lambda boundary!'.format(min_lam,fname))

dr = dr_tco
for hyper_w in [True]:#,False]:
    if hyper_w:
        # hyper-weights fit
        Zoutfile = os.path.join(outdir,'Zout{}_hyper-w.csv'.format(suffix,f_beta))
        Goutfile = os.path.join(outdir,'Gout{}_hyper-w.csv'.format(suffix,f_beta))
        pkl = os.path.join(outdir,'obj{}_hyper-w.pkl'.format(suffix,f_beta))

        dr.ridge_fit(freq,Zc,hyper_weights=True,hyper_lambda=False,hw_wbar='unity',hw_beta=1e10,
                     lambda_0=min_lam,**fit_kw)

        # save results
        Z_pred = dr.predict_Z(freq)
        Z_res = pd.DataFrame(np.array([freq,Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
        Z_res['sigma_re'] = 1/dr.distribution_fits['DRT']['weights'].real**0.5
        Z_res['sigma_im'] = 1/dr.distribution_fits['DRT']['weights'].imag**0.5

        g_pred = dr.predict_distribution('DRT',eval_tau=tau_plot_tco)
        g_res = pd.DataFrame(np.array([tau_plot_tco,g_pred]).T,
                             columns=['tau','gamma'])
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

        # pickle the drt object for access to all other attributes
        save_pickle(dr,pkl)
    else:
        # hyper-lambda fit
        f_betas = [1e-2,1e-1,1,1e1,1e2]
        for f_beta in f_betas:
            Zoutfile = os.path.join(outdir,'Zout{}_fbeta={:.3g}.csv'.format(suffix,f_beta))
            Goutfile = os.path.join(outdir,'Gout{}_fbeta={:.3g}.csv'.format(suffix,f_beta))
            pkl = os.path.join(outdir,'obj{}_fbeta={:.3g}.pkl'.format(suffix,f_beta))

            dr.ridge_fit(freq,Zc,part='both',lambda_0=min_lam,hl_fbeta=f_beta,**fit_kw)

            # save results
            Z_pred = dr.predict_Z(freq)
            Z_res = pd.DataFrame(np.array([freq,Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])

            g_pred = dr.predict_distribution('DRT',eval_tau=tau_plot_tco)
            g_res = pd.DataFrame(np.array([tau_plot_tco,g_pred]).T,
                                 columns=['tau','gamma'])
            Z_res.to_csv(Zoutfile,index=False)
            g_res.to_csv(Goutfile,index=False)

            # pickle the drt object for access to all other attributes
            save_pickle(dr,pkl)


Dumped pickle to results\obj_PDAC_hyper-w.pkl
